# Converting a Normal EPANET .inp File to a SWMM .inp file assuming Flow-Restricted Withdrawal (FRW)
This notebook takes an input EPANET file with demands input normally as a CWS base demand and outputs a .inp file configured for SWMM and uses a FRW assumption  
This conversion is a modified version of the method presented by Campisano et al. (2018) [1] and posited by Cabrera-Bejar & Tzatchkov (2009) [2]  
A simplified schematic of the modified demand node in this method (SWMM-FR) is seen below:  

  ![](SWMM-FR.png) 

### First, we import the necessary libraries and packages
**WNTR** for building EPANET network models in Python  
**NUMPY & PANDAS** for data handling and processing  
**re** for searching and matching text in the .inp file using regular expressions

In [21]:
import wntr
import numpy as np 
import pandas as pd
import re
import math

### Specifying paths for EPANET.inp File to be Converted and preprocessing the input
**Warning:** *Paths in this script (and the rest of this repo) are absolute unless running the network files provided within the repo*  
Input filename (with extensions) as string.  
For running the .inp files in this repository, you can use this relative path `"../Network-Files/Network X/"` where X is the network number 

In [22]:
 # Replace with appropriate path and filename
directory='/Users/omaraliamer/Desktop/UofT/Publications/How to Model IWS/Github/IWS-Modelling-Methods-Repo/Network-Files/Network 3/'
filename='Network3_12hr_PDA.inp'
name_only=filename[0:-7]
print("Selected File: ",name_only)
abs_path=directory+filename

Selected File:  Network3_12hr_


### Necessary Assumptions Input
Converting a CWS demand-driven analysis into an IWS pressure-driven analysis requires some assumptions in all methods  
The resistance of the service connection between the demand junction and the household (end-user) is uncertain and is modelled using two assumptions  
The **desired head (pressure)** is the pressure at the demand junction at which (or above) the consumer can satisfy their full demand in the supply duration (or possible less)  
The **minimum head (pressure)** is the minimum pressure at the demand junction required for flow to begin passing through the service connection  
These two assumptions dictate the flow-pressure relationship that determines the pressure-dependent flow through the service connection as follows:

$$ Q\, = \!Q_{des}\sqrt{\frac{H_{j}-H^{min}}{H^{des}-H^{min}}} \quad[1]$$ 
Where Q is the flow through the service connection, $Q_{des}$ is the desired (base) demand, $H_j$ is the head at the demand junction $j$, $H^{min}$ is the minimum head, and $H^{des}$ is the desired head


In [23]:
desired_pressure=10     # Set the desired pressure
minimum_pressure=0      # Set the minimum pressure
pressure_diff=desired_pressure-minimum_pressure  

### Extracting information from the EPANET file
To modify the .inp file, demand junction IDs, elevations, x and y coordinates  
We use wntr to build the network model of the input file and use wntr's junctions module to extract the details of each node

In [24]:
demand_nodes=[]       # For storing list of nodes that have non-zero demands
desired_demands=[]    # For storing demand rates desired by each node for desired volume calculations
elevations=[]         # For storing elevations of demand nodes
coords=dict()         # For storing coordinates corresponding to each node as a tuple with the id as key
all_nodes=[]          # For storing list of node ids of all nodes
all_elevations=[]     # For storing elevations of all nodes
## MAYBE SAVE ALL NODE IDS IN DATAFRAME WITH ELEVATION AND BASE DEMAND AND THEN FILTER DATA FRAME LATER FOR DEMAND NODES ONLY

# Creates a network model object using EPANET .inp file
network=wntr.network.WaterNetworkModel(abs_path)

# Iterates over the junction list in the Network object
for node in network.junctions():
    all_nodes.append(node[1].name)
    all_elevations.append(node[1].elevation)
    coords[node[1].name]=node[1].coordinates
    # For all nodes that have non-zero demands
    if node[1].base_demand != 0:
        # Record node ID (name), desired demand (base_demand) in CMS, elevations, x and y coordinates
        demand_nodes.append(node[1].name)
        desired_demands.append(node[1].base_demand)
        elevations.append(node[1].elevation)
        

conduit_ids= []
conduit_from= []
conduit_to= []
conduit_lengths= []
conduit_diameters= []
for link in network.links():
    conduit_ids.append(link[1].name)
    conduit_from.append(link[1].start_node_name)
    conduit_to.append(link[1].end_node_name)
    conduit_lengths.append(link[1].length)
    conduit_diameters.append(link[1].diameter)

reservoir_ids=[]
reservoir_heads={}
reservoir_coords={}
for reservoir in network.reservoirs():
    reservoir_ids.append(reservoir[1].name)
    reservoir_heads[reservoir_ids[-1]]=reservoir[1].base_head
    reservoir_coords[reservoir_ids[-1]]=reservoir[1].coordinates


# Get the supply duration in minutes (/60) as an integer
supply_duration=int(network.options.time.duration/60)

### Writing Junctions


In [25]:
MaxDepth=[0]*len(all_nodes)
InitDepth=MaxDepth
SurDepth=[100] * len(all_nodes)  # High value to prevent surcharging
Aponded=MaxDepth

orig_junctions=pd.DataFrame(list(zip(all_nodes,all_elevations,MaxDepth,InitDepth,SurDepth,Aponded)))
orig_junctions=orig_junctions.to_string(header=False,index=False,col_space=10).splitlines()

### Writing Storage Nodes


In [26]:
reservoir_elevations=[0]*len(reservoir_ids)
MaxDepth=[max(100,max(reservoir_heads.values())+10)]*len(reservoir_ids)
InitDepth=reservoir_heads.values()
reservoir_shape=["FUNCTIONAL"]*len(reservoir_ids)
reservoir_coeff=[0]*len(reservoir_ids)
reservoir_expon=[0]*len(reservoir_ids)
reservoir_const=[1000000]*len(reservoir_ids)
reservoir_fevap=reservoir_expon
reservoir_psi=reservoir_fevap

storage_section=pd.DataFrame(zip(reservoir_ids,reservoir_elevations,MaxDepth,InitDepth,reservoir_shape,reservoir_coeff,reservoir_expon,reservoir_const,reservoir_fevap,reservoir_psi))
storage_section=storage_section.to_string(header=False,index=False,col_space=10).splitlines()

### Writing Outfalls

In [27]:
outfall_ids=["Outfall"+str(id) for id in desired_demands]
outfall_elevations=elevations
outfall_type=["FREE"]*len(outfall_ids)
stage_data=["   "]*len(outfall_ids)
outfall_gated=["NO"]*len(outfall_ids)

outfall_section=pd.DataFrame(zip(outfall_ids,outfall_elevations,outfall_type,stage_data,outfall_gated))
outfall_section=outfall_section.to_string(header=False,index=False,col_space=10).splitlines()

### Writing Outlets


In [28]:
outlet_ids = ["Outlet"+id for id in demand_nodes]
outlet_from = demand_nodes
outlet_to = outfall_ids
outlet_offset=[0]*len(outlet_ids)
outlet_type=["TABULAR/DEPTH"]*len(outlet_ids)
outlet_qtable=[str(round(demand*1000000)) for demand in desired_demands]  # To generate unique Table IDs for each demand rate (not demand node) i.e., juncitons with the same demand are assigned the same outlet curve
outlet_expon=["    "]*len(outlet_ids)
outlet_gated=["YES"]*len(outlet_ids)

outlets=pd.DataFrame(list(zip(outlet_ids,outlet_from,outlet_to,outlet_offset,outlet_type,outlet_qtable,outlet_expon,outlet_gated)))
outlets=outlets.to_string(header=False,index=False,col_space=10).splitlines()

### Writing Outlet Curves
Example:  
60               Rating     0          0           
60                          2          0.026832816  
60                          4          0.037947332  
60                          6          0.0464758   
60                          8          0.053665631  
60                          10         0.06        
;

In [29]:
table_ids=list(set(outlet_qtable))   # removes duplicates from list
curves_name=[]
curves_type=[]
curves_x=[]
curves_y=[]
for table in table_ids:
    demand=int(table)/1000                # in LPS
    for depth in np.arange(0,11,1):
        curves_name.append(table)
        if depth==0:
            curves_type.append("Rating")
        else: curves_type.append(" ")
        curves_x.append(depth)
        curves_y.append(demand*np.sqrt((depth-minimum_pressure)/(desired_pressure-minimum_pressure)))
    curves_name.append(";")
    curves_type.append(" ")
    curves_x.append(" ")
    curves_y.append(" ")

curves=pd.DataFrame(list(zip(curves_name,curves_type,curves_x,curves_y)))
curves=curves.to_string(header=False,index=False,col_space=10).splitlines()


### INTERPOLATING 
$$ N_{parts}= \left\lceil \frac{L_{pipe}}{\Delta x_{max}} \right\rceil\\
L_{part}=\frac{L_{pipe}}{N_{parts}}\\
\Delta E = \frac{ E_{end}-E_{start}}{N_{parts}}

In [30]:
# Maximum length of conduit allowed
maximum_xdelta=10
junctions=pd.DataFrame(zip(all_nodes,all_elevations,coords.values()),columns=["ID","Elevation","Coordinates"])
junctions.set_index("ID",inplace=True)

conduits=pd.DataFrame(zip(conduit_ids,conduit_from,conduit_to,conduit_lengths,conduit_diameters),columns=["ID","from node","to node","Length","diameter"])
conduits.set_index("ID",inplace=True)

# Loop over each conduit in the original file
for conduit in conduits.index:

    length=conduits["Length"][conduit]  #Stores the length of the current conduit for shorthand

    # If the conduit is bigger than the maximum allowable length (delta x), we will break it down into smaller pipes
    if length>maximum_xdelta:
        # Number of smaller pipes is calculated from 
        n_parts=math.ceil(length/maximum_xdelta)
        # Calculate the length of each part 
        part_length=length/n_parts
        # Start node ID (for shorthand)
        start_node=conduits["from node"][conduit]
        # End node ID (for shorthand)
        end_node=conduits["to node"][conduit]
        # If the start node is a reservoir
        if start_node in reservoir_ids:
            # MAke the start elevation the same as the end but add 1 (since reservoirs don't have ground elevation in EPANET)
            start_elevation=end_elevation+1
        # Otherwise make the start elevation equal to the elevation of the start node
        else: start_elevation=junctions.at[start_node,"Elevation"]
        
        # If the end node is a reservoir
        if end_node in reservoir_ids:
            # MAke the end elevation the same as the start but subtract 1 (since reservoirs don't have ground elevation in EPANET)
            end_elevation=start_elevation-1
        # Make the end elevation equal to the elevation of the end node
        else: end_elevation=junctions.at[end_node,"Elevation"]
        # Calculate the uniform drop (or rise) in elevation for all the intermediate nodes about to be created when this pipe is broken into several smaller ones
        unit_elev_diff=(end_elevation-start_elevation)/n_parts


        if start_node in reservoir_ids:
            start_x=reservoir_coords[start_node][0]
            start_y=reservoir_coords[start_node][1]
        else:
            start_x=junctions.at[start_node,"Coordinates"][0]
            start_y=junctions.at[start_node,"Coordinates"][1]
        

        if end_node in reservoir_ids:
            end_x=reservoir_coords[end_node][0]
            end_y=reservoir_coords[end_node][1]
        else:
            end_x=junctions.at[end_node,"Coordinates"][0]
            end_y=junctions.at[end_node,"Coordinates"][1]
            

        unit_x_diff=(end_x-start_x)/n_parts
        unit_y_diff=(end_y-start_y)/n_parts


# THIS LOOP GENERATES THE SMALLER PIPES TO REPLACE THE ORIGINAL LONG PIPE
        # For each part to be created
        for part in np.arange(1,n_parts+1):

            # CREATING THE LINKS
            # Create the ID for the new smaller pipe as OriginPipeID-PartNumber
            new_id=conduit+"-"+str(part)
            # Set the new pipe's diameter equal to the original one
            conduits.at[new_id,"diameter"]=conduits["diameter"][conduit]
            # Set the start node as OriginStartNode-NewNodeNumber-OriginEndNode  as in the first intermediate nodes between node 13 and 14 will be named 13-1-14
            conduits.at[new_id,"from node"]=start_node+"-"+str(part-1)+"-"+end_node
            # if this is the first part, use the original start node 
            if part==1:
                conduits.at[new_id,"from node"]=start_node
            # Set the end node as OriginStartNode-NewNodeNumber+1-OriginEndNode  as in the second intermediate nodes between node 13 and 14 will be named 13-2-14
            conduits.at[new_id,"to node"]=start_node+"-"+str(part)+"-"+end_node
            # If this is the last part, use the original end node as the end node
            if part==n_parts:
                conduits.at[new_id,"to node"]=end_node
            # Set the new pipe's length to the length of each part
            conduits.at[new_id,"Length"]=part_length

            if part<n_parts:
                junctions.at[conduits.at[new_id,"to node"],"Elevation"]=start_elevation+part*unit_elev_diff
                junctions.at[conduits.at[new_id,"to node"],"Coordinates"]=(start_x+part*unit_x_diff,start_y+part*unit_y_diff)

    # After writing the new smaller pipes, delete the original pipe (redundant)
    conduits.drop(conduit,inplace=True)
            
            


### Writing Conduit Section
NAME    FROM NODE   TO NODE   Length   Roughness    InOffset     OutOffset     InitFlow    MaxFlow

In [31]:
manning=[0.011]*len(conduits.index)
InOffset=[0]*len(conduits.index)
OutOffset=InOffset
InitFlow=InOffset
MaxFlow=InOffset

conduit_section=pd.DataFrame(zip(conduits.index,conduits["from node"],conduits["to node"],conduits["Length"],manning,InOffset,OutOffset,InitFlow,MaxFlow))
conduit_section=conduit_section.to_string(index=False,header=False,col_space=10).splitlines()

### Writing X-Sections
Link    Shape     Geom1 (DIAMETER)    Geom2 (HW Coefficient)     Geom3    Geom 4       Barrels     Culvert (EMPTY)

In [32]:
shape=["FORCE_MAIN"]*len(conduits.index)
hwcoeffs=[130]*len(shape)
geom3=[0]*len(shape)
geom4=geom3
nbarrels=[1]*len(shape)

xsections_section=pd.DataFrame(zip(conduits.index,shape,conduits["diameter"],hwcoeffs,geom3,geom4,nbarrels))
xsections_section=xsections_section.to_string(header=False,index=False, col_space=10).splitlines()